In [ ]:
import os
from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import pandas as pd

zip_path = '/content/drive/MyDrive/коменты хак/dataset.jsonl.zip'

extract_path = '/content/drive/MyDrive/коменты хак/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


FileNotFoundError: ignored

In [ ]:
import pandas as pd
data = pd.read_json('/content/drive/MyDrive/dataset.jsonl',lines=True)
df = pd.DataFrame(data)

In [ ]:
!python -m spacy download ru_core_news_md  # Или 'ru_core_news_lg'



2023-12-14 20:06:35.295553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 20:06:35.295618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 20:06:35.297358: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 20:06:36.993405: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 9.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [ ]:
import pandas as pd

batch_size = 1000
batches = [df[i:i + batch_size] for i in range(0, df.shape[0], batch_size)]

processed_posts = []
processed_comments = []

for batch in batches:
    posts = batch[batch['root_id'].isna()]
    comments = batch[~batch['root_id'].isna()]

    processed_posts_batch = posts[['text', 'url', 'hash', 'id', 'date']].copy()

    processed_comments_batch = comments[['text', 'url', 'hash', 'id', 'date', 'root_id', 'parent_id']].copy()

    processed_posts.append(processed_posts_batch)
    processed_comments.append(processed_comments_batch)

df_processed_posts = pd.concat(processed_posts, ignore_index=True)
df_processed_comments = pd.concat(processed_comments, ignore_index=True)

all_comments = df_processed_comments.groupby('root_id')['text'].apply(lambda x: ' '.join(x)).reset_index()
all_comments_hash = df_processed_comments.groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()
all_comments_summary = pd.merge(all_comments, all_comments_hash, on='root_id', how='left')
all_comments_summary = pd.merge(all_comments_summary, df_processed_posts[['id', 'text']], left_on='root_id', right_on='id', how='left')
all_comments_summary = all_comments_summary.rename(columns={'id': 'post_id', 'text_x': 'comments', 'text_y': 'post_text', 'hash': 'comment_hashes'})
all_comments_summary = all_comments_summary[['post_id', 'comments', 'post_text', 'comment_hashes']]

post_comments = df_processed_comments[~df_processed_comments['parent_id'].notna()]
post_comments_hash = df_processed_comments[~df_processed_comments['parent_id'].notna()].groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()
post_comments_summary = pd.merge(post_comments, post_comments_hash, on='root_id', how='left')
post_comments_summary = pd.merge(post_comments_summary, df_processed_posts[['id', 'text']], left_on='root_id', right_on='id', how='left')
post_comments_summary = post_comments_summary.rename(columns={'id': 'post_id', 'text_x': 'comments', 'text_y': 'post_text', 'hash': 'comment_hashes'})

topic_comments = df_processed_comments[df_processed_comments['parent_id'].notna()]
topic_comments_hash = df_processed_comments[df_processed_comments['parent_id'].notna()].groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()
topic_comments_summary = pd.merge(topic_comments, topic_comments_hash, on='root_id', how='left')
topic_comments_summary = pd.merge(topic_comments_summary, df_processed_posts[['id', 'text']], left_on='root_id', right_on='id', how='left')
topic_comments_summary = topic_comments_summary.rename(columns={'id': 'post_id', 'text_x': 'comments', 'text_y': 'post_text', 'hash': 'comment_hashes'})
#topic_comments_summary = topic_comments_summary[['post_id', 'comments', 'post_text', 'comment_hashes']]

#print(all_comments_summary)
#print(post_comments_summary)
#print(topic_comments_summary)


<ipython-input-71-c2f289844147>:41: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  post_comments_hash = df_processed_comments[~df_processed_comments['parent_id'].notna()].groupby('root_id')['hash'].apply(lambda x: ', '.join(x)).reset_index()


In [ ]:
topic_comments_summary=topic_comments_summary[:50]

In [ ]:
df=df[:50]

In [ ]:
import pandas as pd
import spacy
from concurrent.futures import ThreadPoolExecutor

nlp = spacy.load('ru_core_news_md')

topic_comments_summary = topic_comments_summary.dropna(subset=['comments', 'post_text'])

def calculate_similarity(row):
    comment_doc = nlp(row['comments'])
    post_text_doc = nlp(row['post_text'])

    if comment_doc.vector_norm and post_text_doc.vector_norm:
        return comment_doc.similarity(post_text_doc)
    else:
        return 0.0

with ThreadPoolExecutor() as executor:
    topic_comments_summary['cosine_similarity'] = list(executor.map(calculate_similarity, topic_comments_summary[['comments', 'post_text']].to_dict(orient='records')))

thematic_comments = topic_comments_summary[topic_comments_summary['cosine_similarity'] > topic_comments_summary['cosine_similarity'].median()]
indirect_comments = topic_comments_summary[(topic_comments_summary['cosine_similarity'] >= topic_comments_summary['cosine_similarity'].quantile(0.1)) & (topic_comments_summary['cosine_similarity'] <= topic_comments_summary['cosine_similarity'].median())]

print("Thematic Comments:")
print(thematic_comments.head())

print("\nIndirect Comments:")
print(indirect_comments.head())


Thematic Comments:
                                            comments  \
1                        лучше смартфона нету ничего   
2  Супер контент!Годные видосы и крутой канал.Как...   
6                        Говорящий палец получается?   
7  Ах да. Автор из Украины, так что для вас это д...   
9                        Пончики такие же как в ярче   

                                                 url  \
1  http://www.youtube.com/watch?v=3BkJqEg7JgY&lc=...   
2  http://www.youtube.com/watch?v=RZjVWF0lJCA&lc=...   
6  http://www.youtube.com/watch?v=i9SkM12kPlo&lc=...   
7  http://www.youtube.com/watch?v=pRAU_YpVBac&lc=...   
9  http://www.youtube.com/watch?v=waCVwtK3AMQ&lc=...   

                             hash_x  \
1  8e4a671ea0e7aadb5f56477696f52a84   
2  4eebeb98c7a99b91ad6cf49d8c541de7   
6  e42fdf88d0382318940c8ac51dcf3e5d   
7  23b4fb997b6fd4428603f75a84c4f1df   
9  a46f2327e20288b985f51a2129eabc9a   

                                                id_x                date

In [ ]:
indirect_comments.head()

,comments,url,hash_x,id_x,date,root_id,parent_id,hash_y,id_y,post_text,cosine_similarity
0,Спасибо),http://www.youtube.com/watch?v=NaQVXXoMJk0&lc=...,6efc18ae0c594bb9210985b8b72d914d,UgykYD7CA7PTXrYf7HF4AaABAg.9welsmcr4b79wenwXhba33,2023-11-03 17:00:00,NaQVXXoMJk0,UgykYD7CA7PTXrYf7HF4AaABAg,"6efc18ae0c594bb9210985b8b72d914d, 10d2c12f4cf2...",NaQVXXoMJk0,на какое звание играю ? #on #pyf #so2 #нож #ди...,0.000000
3,Спасибо 😊,http://www.youtube.com/watch?v=f9DuWhE9aSw&lc=...,e62fc92f3b79a4750874f5c373c8ee21,UgwpEpDt8AvkVZoDZ4R4AaABAg.9wcbXWYIAgk9weGRUkh_E9,2023-11-03 12:00:00,f9DuWhE9aSw,UgwpEpDt8AvkVZoDZ4R4AaABAg,"e62fc92f3b79a4750874f5c373c8ee21, 6681588f1fa2...",f9DuWhE9aSw,"США ввела новые санкции, Что будет с Питерской...",0.000000
4,🇧🇾🇧🇾🇧🇾👍,http://www.youtube.com/watch?v=asfawPoOASQ&lc=...,01c6601d7170e6799492dec98261cb5e,UgyYp3N75iN-vaPP64V4AaABAg,2023-11-03 20:00:00,asfawPoOASQ,asfawPoOASQ,"01c6601d7170e6799492dec98261cb5e, bb1bed5797b9...",asfawPoOASQ,БелАЭС самый масштабный стратегический проект ...,0.000000
5,Миш просто знай ты даже лучше чем бист ты топ...,http://www.youtube.com/watch?v=9xxyy5LgKco&lc=...,cdce34bacc61a97e22f74b525fcce4aa,UgyMJvjCRbveYu1mlrx4AaABAg,2023-11-03 20:36:00,9xxyy5LgKco,9xxyy5LgKco,"cdce34bacc61a97e22f74b525fcce4aa, 00a4393484ac...",9xxyy5LgKco,Литвин ЧЕЛЛЕНДЖ ЗА КАКУЮ СУММУ ГОТОВЫ УВОЛИТЬС...,-0.020509
16,"0,5годик",http://www.youtube.com/watch?v=dOG1bW1v1Ks&lc=...,ed01df43fe02bed28fb2f316b21cc206,UgyTSE8T4vTinYZ6k4N4AaABAg,2023-11-03 12:00:00,dOG1bW1v1Ks,dOG1bW1v1Ks,"ed01df43fe02bed28fb2f316b21cc206, c6ed20c22b05...",dOG1bW1v1Ks,сколько мне лет ?,0.000000


In [ ]:
thematic_comments.head()

,comments,url,hash_x,id_x,date,root_id,parent_id,hash_y,id_y,post_text,cosine_similarity
1,лучше смартфона нету ничего,http://www.youtube.com/watch?v=3BkJqEg7JgY&lc=...,8e4a671ea0e7aadb5f56477696f52a84,Ugyzw39qUWt0wZMAQu14AaABAg.9we3jXhmP2d9wh07Fkwxw1,2023-11-04 13:58:00,3BkJqEg7JgY,Ugyzw39qUWt0wZMAQu14AaABAg,"8e4a671ea0e7aadb5f56477696f52a84, 0fa680aaf8fa...",3BkJqEg7JgY,🔥Мой telegram - https://t.me/KMreview<br />\n<...,0.586885
2,Супер контент!Годные видосы и крутой канал.Как...,http://www.youtube.com/watch?v=RZjVWF0lJCA&lc=...,4eebeb98c7a99b91ad6cf49d8c541de7,UgxEv8ef0ihBOk5l57l4AaABAg,2023-11-02 21:00:00,RZjVWF0lJCA,RZjVWF0lJCA,"4eebeb98c7a99b91ad6cf49d8c541de7, 50b8a6bb9183...",RZjVWF0lJCA,Игра 18+) Игра Hellblade: Senua’s Sacrifice ⭐<...,0.385001
6,Говорящий палец получается?,http://www.youtube.com/watch?v=i9SkM12kPlo&lc=...,e42fdf88d0382318940c8ac51dcf3e5d,UgyWd91vEzFFrhumkA94AaABAg,2023-11-03 17:12:00,i9SkM12kPlo,i9SkM12kPlo,"e42fdf88d0382318940c8ac51dcf3e5d, 2250b270533f...",i9SkM12kPlo,ПЕСОК для БЕТОНА,0.578772
7,"Ах да. Автор из Украины, так что для вас это д...",http://www.youtube.com/watch?v=pRAU_YpVBac&lc=...,23b4fb997b6fd4428603f75a84c4f1df,UgzsVy6-_9hgdCSINzN4AaABAg.9weOmVoOIhI9weQ_X3Eir0,2023-11-03 14:00:00,pRAU_YpVBac,UgzsVy6-_9hgdCSINzN4AaABAg,"23b4fb997b6fd4428603f75a84c4f1df, 62562bcc11db...",pRAU_YpVBac,Для получения текущей даты или произвольной да...,0.811687
9,Пончики такие же как в ярче,http://www.youtube.com/watch?v=waCVwtK3AMQ&lc=...,a46f2327e20288b985f51a2129eabc9a,UgytXjX_5w_pPQQl5_14AaABAg,2023-11-03 15:00:00,waCVwtK3AMQ,waCVwtK3AMQ,"a46f2327e20288b985f51a2129eabc9a, 4b6034e500f0...",waCVwtK3AMQ,11.11 в Чизкейк.ру: успевай на самую выгодную ...,0.726710


In [ ]:
df=df[:10]

In [ ]:
all_comments_summary=all_comments_summary[:100]

In [ ]:
!pip install rouge_score googletrans -qU

In [ ]:
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install hugginface_hub
from huggingface_hub import notebook_login
notebook_login()

ERROR: Could not find a version that satisfies the requirement hugginface_hub (from versions: none)
ERROR: No matching distribution found for hugginface_hub


In [ ]:
!pip install torch

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 625.5 kB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
import concurrent.futures
import time
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer


MODEL_NAME = 'cointegrated/rut5-base-absum'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TARGET_TIME_PER_BATCH = 2.0


def initialize_model():
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    return model, tokenizer


def summarize(text, model, tokenizer, n_words=20, max_length=10000, num_beams=2, do_sample=True, repetition_penalty=20.0, **kwargs):
    if n_words:
        text = f'[{n_words}] {text}'
    inputs = tokenizer(text, return_tensors='pt', padding=True).to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            **kwargs
        )

    return tokenizer.decode(out[0], skip_special_tokens=True)


def summarize_batch(batch, model, tokenizer, cache):
    return [cache[text] if text in cache else cache.setdefault(text, summarize(text, model, tokenizer)) for text in batch]


def process_batch(batch, model, tokenizer, cache):
    return summarize_batch(batch, model, tokenizer, cache)


def add_summary_column(df, column='comments', summary_column='summary_comments', batch_size=256):
    model, tokenizer = initialize_model()
    data = df[column]

    cache = {}

    num_batches = -(-len(data) // batch_size)
    start_time = time.time()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i in range(num_batches):
            start_index = i * batch_size
            end_index = min((i + 1) * batch_size, len(data))
            batch_data = data[start_index:end_index]


            future_data = executor.submit(process_batch, batch_data, model, tokenizer, cache.copy())

            futures.append(future_data)


        for future in concurrent.futures.as_completed(futures):
            results = future.result()
            start_index = futures.index(future) * batch_size
            end_index = start_index + len(results)


            df.loc[start_index:end_index - 1, summary_column] = results


            elapsed_time = time.time() - start_time
            batch_number = len(futures)
            time_per_batch = elapsed_time / batch_number if batch_number > 0 else 0.0
            print(f"Batch {batch_number}/{num_batches}, Time per batch: {time_per_batch:.4f} seconds")

            if elapsed_time < TARGET_TIME_PER_BATCH * batch_number:
                time.sleep(TARGET_TIME_PER_BATCH * batch_number - elapsed_time)

    total_time = time.time() - start_time
    print(f"Total time taken: {total_time:.2f} seconds")

    return df

# Example usage
all_comments_summary = add_summary_column(all_comments_summary, column='comments', summary_column='summary_comments')


Batch 1/1, Time per batch: 10.2390 seconds
Total time taken: 10.24 seconds


<ipython-input-48-16098d2d6eca>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[start_index:end_index - 1, summary_column] = results


In [ ]:
all_comments_summary=all_comments_summary[:10]

In [ ]:
import pandas as pd
from bert_score import score
import numpy as np



# Handle NaN values
all_comments_summary['summary_comments'].fillna("", inplace=True)
all_comments_summary['post_text'].fillna("", inplace=True)

# Extracting the text from the DataFrame
references = all_comments_summary['summary_comments'].tolist()
hypotheses = all_comments_summary['post_text'].tolist()

# Calculate BERT_score
P, R, F1 = score(hypotheses, references, lang="ru", rescale_with_baseline=True)

# Create a new DataFrame with BERT_score results
bert_score_df = pd.DataFrame({'Precision': P.tolist(), 'Recall': R.tolist(), 'F1 Score': F1.tolist()})

# Display the BERT_score results DataFrame
print(bert_score_df)


<ipython-input-49-443f2c2e8b58>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_comments_summary['summary_comments'].fillna("", inplace=True)
<ipython-input-49-443f2c2e8b58>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_comments_summary['post_text'].fillna("", inplace=True)


   Precision    Recall  F1 Score
0   0.573869  0.650384  0.609735
1   0.345360  0.642797  0.449314
2   0.586937  0.651146  0.617377
3   0.398450  0.581711  0.472948
4   0.481742  0.596896  0.533172
5   0.610041  0.676362  0.641492
6   0.401029  0.699475  0.509784
7   0.398884  0.631473  0.488927
8   0.442259  0.641253  0.523483
9   0.537944  0.656513  0.591344


In [ ]:
!pip install sentencepiece


In [ ]:
all_comments_summary=all_comments_summary[0:30]

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5f381fcf82b2efd5519b3560fd38a6ad6aee63a613bd9cdcb8601a70b7b26b66
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
all_comments_summary=all_comments_summary[:10]

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from concurrent.futures import ThreadPoolExecutor

MODEL_NAME = 'cointegrated/rut5-base-absum'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def initialize_model():
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    return model, tokenizer

def summarize(text, model, tokenizer, n_words=3, max_length=10000, num_beams=2, do_sample=False, repetition_penalty=10.0, **kwargs):
    if n_words:
        text = f'[{n_words}] {text}'
    inputs = tokenizer(text, return_tensors='pt', padding=True).to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            **kwargs
        )

    return tokenizer.decode(out[0], skip_special_tokens=True)


def summarize_batch(batch, model, tokenizer, cache):
    results = []
    for text in batch:
        if text not in cache:
            summary = summarize(text, model, tokenizer)
            cache[text] = summary
        else:
            summary = cache[text]
        results.append(summary)

    torch.cuda.empty_cache()

    return results

def add_summary_column(df, comment_column='post_text', summary_column='summary_post', batch_size=8):
    model, tokenizer = initialize_model()
    comments = df[comment_column]

    cache = {}

    with ThreadPoolExecutor() as executor:
        batches = [comments[i:i + batch_size] for i in range(0, len(comments), batch_size)]
        results = list(executor.map(lambda batch: summarize_batch(batch, model, tokenizer, cache), batches))

    summaries = [item for sublist in results for item in sublist]

    df = df.copy()
    df.loc[:, summary_column] = summaries

    torch.cuda.empty_cache()

    return df


all_comments_summary = add_summary_column(all_comments_summary, comment_column='post_text', summary_column='summary_post')


In [ ]:
from rouge_score import rouge_scorer
import pandas as pd

# Initializing T5 model and tokenizer
def initialize_model():
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    return model, tokenizer

# Summarization function
def summarize(text, model, tokenizer, n_words=3, max_length=10000, num_beams=2, do_sample=False, repetition_penalty=10.0, **kwargs):
    if n_words:
        text = f'[{n_words}] {text}'
    inputs = tokenizer(text, return_tensors='pt', padding=True).to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            **kwargs
        )

    return tokenizer.decode(out[0], skip_special_tokens=True)

# Summarization for batches
def summarize_batch(batch, model, tokenizer, cache):
    results = []
    for text in batch:
        if text not in cache:
            summary = summarize(text, model, tokenizer)
            cache[text] = summary
        else:
            summary = cache[text]
        results.append(summary)

    # Clear GPU cache after processing each batch to release memory
    torch.cuda.empty_cache()

    return results

# Adding summary column for a single column 'comments'
def add_summary_column(df, comment_column='post_text', summary_column='summary_post', batch_size=8):
    model, tokenizer = initialize_model()
    comments = df[comment_column]

    cache = {}  # Cache for storing summaries

    with ThreadPoolExecutor() as executor:
        batches = [comments[i:i + batch_size] for i in range(0, len(comments), batch_size)]
        results = list(executor.map(lambda batch: summarize_batch(batch, model, tokenizer, cache), batches))

    summaries = [item for sublist in results for item in sublist]

    # Use .copy() to avoid SettingWithCopyWarning
    df = df.copy()
    df.loc[:, summary_column] = summaries

    # Clear GPU cache to release unnecessary data
    torch.cuda.empty_cache()

    return df

# Example usage
all_comments_summary = add_summary_column(all_comments_summary, comment_column='post_text', summary_column='summary_post')

# Calculating ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores_list = []

for _, row in all_comments_summary.iterrows():
    reference = row['summary']
    hypothesis = row['summary_post']

    rouge_scores = scorer.score(reference, hypothesis)
    rouge_scores_list.append(rouge_scores)

# Creating a DataFrame with ROUGE scores
rouge_df = pd.DataFrame(rouge_scores_list)
print(rouge_df)


            rouge1           rouge2     rougeL
0  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
1  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
2  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
3  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
4  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
5  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
6  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
7  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
8  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)
9  (0.0, 0.0, 0.0)  (0.0, 0.0, 0.0)  (0, 0, 0)


МЕтрика bert_score

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import dask.dataframe as dd
import time

# Constants
MODEL_NAME = 'cointegrated/rut5-base-absum'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TARGET_TIME_PER_BATCH = 2.0  # Target processing time per batch (in seconds)

# Initializing T5 model and tokenizer
def initialize_model():
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)
    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    return model, tokenizer

# Summarization function
def summarize(text, model, tokenizer, n_words=3, max_length=10000, num_beams=2, do_sample=False, repetition_penalty=10.0, **kwargs):
    if n_words:
        text = f'[{n_words}] {text}'
    inputs = tokenizer(text, return_tensors='pt', padding=True).to(DEVICE)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=num_beams,
            do_sample=do_sample,
            repetition_penalty=repetition_penalty,
            **kwargs
        )

    return tokenizer.decode(out[0], skip_special_tokens=True)

# Summarization for batches
def summarize_batch(batch, model, tokenizer, cache):
    results = []
    for text in batch:
        if text not in cache:
            summary = summarize(text, model, tokenizer)
            cache[text] = summary
        else:
            summary = cache[text]
        results.append(summary)

    return results

# Function for mapping in ThreadPoolExecutor
def process_batch(batch, model, tokenizer, cache):
    return summarize_batch(batch, model, tokenizer, cache)

# Adding summary columns for two input columns and storing results in two new columns
def add_summary_columns(df, column1='comments', column2='post_text', summary_column1='summary_comments', summary_column2='summary_posts', batch_size=8):
    model, tokenizer = initialize_model()
    data1 = df[column1]
    data2 = df[column2]

    cache = {}  # Cache for storing summaries

    # Determine the number of batches based on the target processing time
    num_batches = int(len(data1) / batch_size)
    start_time = time.time()

    # Using Dask for parallel processing
    ddf = dd.from_pandas(df, npartitions=num_batches)

    def process_partition(partition):
        cache = {}  # Local cache for each partition
        results_data1 = []
        results_data2 = []

        for i, row in partition.iterrows():
            summary_data1 = process_batch([row[column1]], model, tokenizer, cache)[0]
            summary_data2 = process_batch([row[column2]], model, tokenizer, cache)[0]

            results_data1.append(summary_data1)
            results_data2.append(summary_data2)

        return pd.DataFrame({summary_column1: results_data1, summary_column2: results_data2})

    processed_df = ddf.map_partitions(process_partition, meta={summary_column1: str, summary_column2: str}).compute()

    total_time = time.time() - start_time
    print(f"Total time taken: {total_time:.2f} seconds")

    return pd.concat([df, processed_df], axis=1)

# Example usage
# Assuming your original DataFrame is named 'original_df'
all_comments_summary = add_summary_columns(all_comments_summary,column1='comments', column2='post_text', summary_column1='summary_comments', summary_column2='summary_posts')


NameError: ignored

In [ ]:
topic_comments_summary=topic_comments_summary[:10]

In [ ]:
all_comments_summary

,post_id,comments,post_text,comment_hashes,summary_comments,summary_posts
0,--SDWklGeAE,@kryxmks. Зная большую часть людей в коммент...,"Приколы,BITTUEV,юмор,вайны,лучшие приколы,кари...","bb746cbc009b15cd6a0f451b102cb31e, 4c8a31f1ef12...","Вот что вы думаете, этот чел просто овощ.","Лучшие приколы,BITTUEV,БИТУЕВ братик,битуев бр..."
1,-100194451_510298,"[id50188945|Сергей], это разве Угол))). Чуть ч...",Штрафы за дрифт начали приходить 🙈<hr/><br/><a...,"7361155a282dda4629d018f07b4c24a9, 959aafd544be...",Смотри на передние колеса. Подумай над этим,Проверки за дрифт в России начались.
2,-100272777_810169,Да нет надо просто с лопатами к Администрации ...,Анонимно пожалуйста куда ещё звонить или писат...,"e95970a83cee2788f8b968b4b6147236, edffa95bbb21...","Если вы не знаете, что это вообще там?",В сёлах не чистят дороги.
3,-100283479_678834,Кошель та с рунами 🤔 Смотря сколько взять... е...,"В магазине Пятерочка ул.Манченко, д.25 найден ...","4be8b6682fb38fbd172c0845fb78e1eb, 8438cd381364...","Открой кошельку с рунами. Попробуй доказать, ч...",В магазине Пятерочка нашли кошелек.
4,-100283479_679838,"автор эстонец? [id22566180|Юрий], ""помню был у...","28.11.2023 , примерно в 20:30 вечера произошла...","a069cc8ada28697069556d997d194a3b, 0904907e4fcd...","Что вы думаете, что у тебя такой броневик, что...",После неприятной ситуации в городе Черняховски...
5,-100345006_751238,Ты подарил мне в день рождения стиральную маши...,#начало или #продолжи_фразу\nТы подарил мне в ...,"130ee0e3b90713eb23bdcdf17f589894, 4832129d64f2...",Подарил мне в день рождения стиральную машинку,Подари мне в день рождения стиральную машинку.
6,-1003715_1457957,Значит ли это что мы не скоро увидим но...,Неизданные песни Майкла Джексона продают на ау...,"927e493be792985883ee93e74147435c, 1cccb105185e...",Если вы не хотите выпустить музыку неизданного...,Музыкальные партии Майкла Джексона продают на ...
7,-100426847_41979,Какой наполнитель? Цена? Ну нах такую красоту ...,Красотки 🥰Любимки наши ❤️<br>А на Северной 76 ...,"38ecc3d99768e2042b678afbd1dac877, bab6c5d6da43...",Цена в графитовом цвете,❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️❤️...
8,-100548814_121765,"[id156309073|Александр], я не против, но ОЧЕНЬ...",🔥 Корейцы есть? 😄 Если у вас есть знакомые / р...,"0aaec9177e87a372d2f048012bf5d25c, d01e0225d0d2...",NaN,NaN
9,-10055384_7436142,"[id91837649|Віктор], 3 вроде даже, одно беспла...",Одна из лучших глав в Alan Wake II чуть не пош...,"dfbb3da272473b9a41334bfdc0ab5d2c, f74acdf7dd41...",NaN,NaN


In [ ]:
### Installation on Colab (As Of Oct)
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_6

In [ ]:
!pip install -U accelerate

  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.25.0.dev0
    Uninstalling accelerate-0.25.0.dev0:
      Successfully uninstalled accelerate-0.25.0.dev0


In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipeline)
quantization_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_compute_dtype=torch.float16,
bnb_4bit_quant_type="nf4",
bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained("Trelis/Mistral-7B-Instruct-v0.1-Summarize-64k")
model = AutoModelForCausalLM.from_pretrained("Trelis/Mistral-7B-Instruct-v0.1-Summarize-64k")

pipeline = pipeline(
"summarization",
model=model_4bit,
tokenizer=tokenizer,
use_cache=True,
device_map="auto",
max_length=500,
do_sample=True,
top_k=5,
num_return_sequences=1,
eos_token_id=tokenizer.eos_token_id,
pad_token_id=tokenizer.eos_token_id,
)
#### Prompt
template = """<s>[INST] You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
Answer the question below from context below :
{context}
{question} [/INST] </s>
"""

question_p = """What is the date for announcement"""
context_p = """ On August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""
prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response

OSError: ignored

In [ ]:
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

mna_news = """JSW Energy on August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore.
Sajjan Jindal led JSW group had signed an exclusivity agreement with Hyderabad based Mytrah Energy to buy the latter’s wind and solar assets.
This is the largest acquisition by JSW Energy since inception, which comprises 17 special purpose vehicles and 1 ancillary SPV. The transaction is subject to approval of the Competition Commission of India (CCI) and other customary approvals standard to a transaction of this size, the company said in a release.
With this acquisition, JSW Energy's current operational generation capacity will go up by over 35 percent from 4,784 MW to 6,537 MW. Currently about 2,500 MW of under-construction wind and hydro projects which are likely to be commissioned in phases over the next 18-24 months, JSW Energy platform capacity gets enhanced to 9.1 GW – where the share of renewables increases to 65 percent, JSW said in a stock exchange filing. Further, this is likely to help the company in achieving its renewable-led capacity growth target of 10 GW by FY25, well ahead of timelines, the firm added.
"""

from langchain.schema.document import Document
documents = [Document(page_content=mna_news, metadata={"source": "local"})]
#######################
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
#######################
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
#######################
retriever = vectordb.as_retriever()
#######################
qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
verbose=True
)

def run_my_rag(qa, query):
print(f"Query: {query}\n")
result = qa.run(query)
print("\nResult: ", result)

### Ask Queries Now
query =""" What company is buyer and seller here """
run_my_rag(qa, query)

In [ ]:
from transformers import pipeline
from rouge_score import rouge_scorer
from googletrans import Translator
import pandas as pd

# Set up the translator
translator = Translator()

# Определите вашу модель для сжатия текста
model = "Gaivoronsky/Mistral-7B-Saiga"
summarizer = pipeline("summarization", model=model)

def is_positive_comment(comment):
    positive_keywords = ["круто", "классно", "здорово"]
    return any(keyword in comment for keyword in positive_keywords)

def filter_toxicity(comment):
    # Реализуйте фильтр токсичности по необходимости
    # В данном случае, предположим, что токсичных комментариев нет
    return comment

def translate_to_russian(comment, src_lang='auto', dest_lang='ru'):
    translation = translator.translate(comment, src=src_lang, dest=dest_lang)
    return translation.text

def summarize(text):
    # Используйте сводную модель напрямую
    summary = summarizer(text, max_length=150, min_length=50, length_penalty=2.0, num_beams=4)
    return summary[0]['summary_text']

# Пример: Загрузите DataFrame с колонкой 'comments'
# Вам нужно заменить 'your_data.csv' на фактическое имя вашего файла данных
df = all_comments_summary
# Примените фильтры и команду summarize к столбцу 'comments'
df['comments'] = df['comments'].apply(filter_toxicity)
df['comments'] = df['comments'].apply(lambda x: x if is_positive_comment(x) else '')
df['comments'] = df['comments'].apply(translate_to_russian)

# Примените команду summarize к столбцу 'comments'
df['summary'] = df['comments'].apply(lambda x: summarize(x))

# Вычислите оценки ROUGE
#rouge_scores = calculate_rouge_scores(df['summary'].tolist(), df['reference_summary'].tolist())
#print(rouge_scores)
